In [2]:
import pandas as pd 
import numpy as np

import warnings
warnings.simplefilter('ignore')

In [3]:
df = pd.read_csv(r"E:\Data Science by SRK\Machine_learning\Classification\Gene_data\cleaned_dataset_Gene.csv")

In [4]:
df

,Gene_One,Gene_Two,Cancer_Present
0,4.3,3.9,1
1,2.5,6.3,0
2,5.7,3.9,1
3,6.1,6.2,0
4,7.4,3.4,1
...,...,...,...
2133,5.0,6.5,1
2134,3.4,6.6,0
2135,2.7,6.5,0
2136,3.3,5.6,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gene_One        2138 non-null   float64
 1   Gene_Two        2138 non-null   float64
 2   Cancer_Present  2138 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 50.2 KB


In [6]:
df.isnull().sum()

Gene_One          0
Gene_Two          0
Cancer_Present    0
dtype: int64

# X and y

In [7]:
X = pd.get_dummies(df.drop('Cancer_Present', axis = 1), drop_first = True)
y = df['Cancer_Present']

# Train_Test_split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = True)

# Modelling and Evaluation

In [9]:
# Random Forest classifire with default parameters
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = True)
model.fit(X_train, y_train)

# Prediction
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)

# Evaluation
from sklearn.metrics import accuracy_score
print("Train accuracy : ", accuracy_score(ypred_train, y_train))
print("Test accuracy : ", accuracy_score(ypred_test, y_test))

from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(model, X, y, cv=5).mean())

Train accuracy :  0.9690058479532164
Test accuracy :  0.8785046728971962
cross_val_score :  0.8769802359430059


# Hyper_parameter Tuning

In [10]:
from sklearn.model_selection import GridSearchCV

# model
estimator = RandomForestClassifier(random_state = True)

# Parameters (which you want to tune and identify the best)
param_grid = {"n_estimators" : list(range(1, 10))}

grid = GridSearchCV(estimator, param_grid, scoring = 'accuracy', cv= 5)

grid.fit(X_train, y_train)

grid.best_params_

{'n_estimators': 8}

# Importance of each features

In [11]:
grid.best_estimator_.feature_importances_


array([0.51180957, 0.48819043])

In [12]:
feats_imp = pd.DataFrame(data = grid.best_estimator_.feature_importances_, 
                         index = X.columns, 
                         columns = ['Feature Importance'])
important_feats = feats_imp[feats_imp['Feature Importance'] > 0]
important_features_list = important_feats.index.to_list()
important_features_list

['Gene_One', 'Gene_Two']

# Final Random Forest model
**with best HPT AND important features**

In [13]:
X_imp = X[important_features_list]

# Train_Test_Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = True)


# Model with best HPT AND important features
final_rf_model = RandomForestClassifier()
final_rf_model.fit(X_train, y_train)

# Prediction
ypred_train = final_rf_model.predict(X_train)
ypred_test = final_rf_model.predict(X_test)

# Evaluation
from sklearn.metrics import accuracy_score
print("Train accuracy : ", accuracy_score(ypred_train, y_train))
print("Test accuracy : ", accuracy_score(ypred_test, y_test))

from sklearn.model_selection import cross_val_score
print("cross_val_score : ", cross_val_score(model, X, y, cv=5).mean())

Train accuracy :  0.9690058479532164
Test accuracy :  0.8808411214953271
cross_val_score :  0.8769802359430059
